# ÔN TẬP: LẬP TRÌNH HƯỚNG ĐỐI TƯỢNG (OOP) TRONG PYTHON

**Thời gian gợi ý:** 9 giờ  
**Mục tiêu**
- Hiểu khái niệm **lớp (class)**, **đối tượng (object)**, **thuộc tính (attribute)**, **phương thức (method)**.
- Ứng dụng OOP vào thiết kế chương trình **có cấu trúc**, dễ mở rộng.
- Nắm các nội dung: `__init__`, `self`, đóng gói, kế thừa, “nạp chồng” theo kiểu Python, trừu tượng.
- Ứng dụng: **mô phỏng quản lý sinh viên**.

> Notebook này gồm: **ôn tập lý thuyết**, **demo minh họa**, **bài tập + lời giải**.


---
## 0) Checklist kiến thức cần nhớ
- Class vs Object
- `__init__`, `self`, instance attribute vs class attribute
- Method: instance / classmethod / staticmethod
- Encapsulation: `_` và `__` (name mangling) + property
- Inheritance: override, `super()`
- Polymorphism: cùng interface - nhiều implementation
- (Giới thiệu) Abstraction: `abc.ABC`, `@abstractmethod`


---
## 1) Khái niệm Class và Object
- **Class (Lớp)**: khuôn mẫu mô tả dữ liệu + hành vi.
- **Object (Đối tượng)**: một instance cụ thể tạo ra từ class.

Ví dụ: lớp `Student` là khuôn, còn `student_a` là một đối tượng.


In [1]:
class Student:
    pass

student_a = Student()
student_b = Student()
student_a, student_b

(<__main__.Student at 0x107894aa0>, <__main__.Student at 0x107d2ec30>)

---
## 2) Constructor `__init__`, `self`, thuộc tính, phương thức
- `__init__` chạy khi tạo đối tượng, dùng để khởi tạo thuộc tính.
- `self` là tham chiếu tới object hiện tại.


In [2]:
class Student:
    def __init__(self, student_id: str, name: str, gpa: float):
        self.student_id = student_id
        self.name = name
        self.gpa = float(gpa)

    def introduce(self) -> str:
        return f"{self.student_id} - {self.name} (GPA={self.gpa:.2f})"

s = Student("SV001", "An", 3.4)
print(s.introduce())

SV001 - An (GPA=3.40)


### Class attribute vs Instance attribute
- **Class attribute**: dùng chung cho mọi object.
- **Instance attribute**: riêng cho từng object.


In [3]:
class Student:
    school = "ITC"  # class attribute

    def __init__(self, student_id, name):
        self.student_id = student_id
        self.name = name

s1 = Student("SV001", "An")
s2 = Student("SV002", "Bình")

print(s1.school, s2.school)
Student.school = "CĐ CNTT TP.HCM"
print(s1.school, s2.school)

ITC ITC
CĐ CNTT TP.HCM CĐ CNTT TP.HCM


---
## 3) Method: instance method, classmethod, staticmethod
- Instance method: nhận `self`
- Classmethod: nhận `cls` (factory method)
- Staticmethod: hàm tiện ích liên quan class


In [4]:
class ScoreUtils:
    @staticmethod
    def clamp_gpa(gpa: float) -> float:
        return max(0.0, min(4.0, float(gpa)))

class Student:
    def __init__(self, student_id, name, gpa):
        self.student_id = student_id
        self.name = name
        self.gpa = ScoreUtils.clamp_gpa(gpa)

    @classmethod
    def from_csv_row(cls, row: str):
        sid, name, gpa = [x.strip() for x in row.split(",")]
        return cls(sid, name, float(gpa))

    def introduce(self):
        return f"{self.student_id} - {self.name} (GPA={self.gpa:.2f})"

s = Student.from_csv_row("SV003, Chi, 4.2")
print(s.introduce())

SV003 - Chi (GPA=4.00)


---
## 4) Đóng gói (Encapsulation)
Python dùng quy ước:
- `_attr`: protected (quy ước)
- `__attr`: name mangling (hạn chế truy cập trực tiếp)

Khuyến nghị: dùng `@property` để kiểm soát dữ liệu + validate.


In [ ]:
class BankAccount:
    def __init__(self, owner: str, balance: float = 0):
        self.owner = owner
        self.__balance = float(balance)

    @property
    def balance(self) -> float:
        return self.__balance

    def deposit(self, amount: float):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Amount must be positive")
        self.__balance += amount

    def withdraw(self, amount: float):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("Amount must be positive")
        if amount > self.__balance:
            raise ValueError("Insufficient funds")
        self.__balance -= amount

acc = BankAccount("Bình", 100)
acc.deposit(50)
acc.withdraw(30)
print(acc.balance)

---
## 5) Kế thừa (Inheritance) & override
- Kế thừa: lớp con nhận thuộc tính/phương thức lớp cha.
- Override: lớp con định nghĩa lại phương thức.
- `super()`: gọi logic lớp cha.


In [ ]:
class Person:
    def __init__(self, name: str):
        self.name = name

    def introduce(self) -> str:
        return f"I am {self.name}."

class Student(Person):
    def __init__(self, name: str, student_id: str):
        super().__init__(name)
        self.student_id = student_id

    def introduce(self) -> str:
        return f"I am {self.name}, student id={self.student_id}."

print(Person("Nam").introduce())
print(Student("Lan", "SV888").introduce())

### “Nạp chồng phương thức” trong Python?
Python không overload theo chữ ký như Java.
Cách thay thế:
- tham số mặc định
- `*args`, `**kwargs`


In [ ]:
class Printer:
    def print_value(self, x=None, y=None):
        if x is None and y is None:
            print("Nothing")
        elif y is None:
            print(f"One value: {x}")
        else:
            print(f"Two values: {x}, {y}")

pr = Printer()
pr.print_value()
pr.print_value(10)
pr.print_value(10, 20)

---
## 6) Đa hình (Polymorphism)
Cùng lời gọi `make_sound()` nhưng hành vi khác nhau tùy lớp.


In [ ]:
class Animal:
    def make_sound(self) -> str:
        raise NotImplementedError

class Dog(Animal):
    def make_sound(self) -> str:
        return "Woof"

class Cat(Animal):
    def make_sound(self) -> str:
        return "Meow"

for a in [Dog(), Cat()]:
    print(a.make_sound())

---
## 7) Trừu tượng (Abstraction) – giới thiệu
Dùng `abc` để ép lớp con phải triển khai phương thức.


In [ ]:
from abc import ABC, abstractmethod

class Repository(ABC):
    @abstractmethod
    def add(self, item): ...
    @abstractmethod
    def list_all(self): ...

class InMemoryRepository(Repository):
    def __init__(self):
        self._items = []
    def add(self, item):
        self._items.append(item)
    def list_all(self):
        return list(self._items)

repo = InMemoryRepository()
repo.add({"id": 1})
print(repo.list_all())

---
# 8) ỨNG DỤNG: MÔ PHỎNG QUẢN LÝ SINH VIÊN

## 8.1 Flowchart (quy trình)
```text
[Nhập lựa chọn menu]
        |
        v
(1) Add Student      -> thêm vào danh sách
(2) Update Student   -> tìm theo id -> cập nhật
(3) Remove Student   -> tìm theo id -> xóa
(4) Search           -> lọc theo keyword
(5) Sort by GPA      -> sắp xếp
(6) Report by Rank   -> thống kê học lực
(0) Exit
```

## 8.2 Code mẫu quản lý sinh viên


In [ ]:
from dataclasses import dataclass
from typing import List, Optional

@dataclass
class Student:
    student_id: str
    name: str
    gpa: float

    def __post_init__(self):
        self.gpa = float(self.gpa)
        if not (0.0 <= self.gpa <= 4.0):
            raise ValueError("GPA must be between 0.0 and 4.0")

    def rank(self) -> str:
        if self.gpa >= 3.6:
            return "Xuất sắc"
        if self.gpa >= 3.2:
            return "Giỏi"
        if self.gpa >= 2.5:
            return "Khá"
        if self.gpa >= 2.0:
            return "Trung bình"
        return "Yếu"

class StudentManager:
    def __init__(self):
        self._students: List[Student] = []

    def add(self, s: Student):
        if self.find_by_id(s.student_id) is not None:
            raise ValueError("Duplicate student_id")
        self._students.append(s)

    def find_by_id(self, student_id: str) -> Optional[Student]:
        for s in self._students:
            if s.student_id == student_id:
                return s
        return None

    def update(self, student_id: str, *, name: Optional[str] = None, gpa: Optional[float] = None) -> bool:
        s = self.find_by_id(student_id)
        if s is None:
            return False
        if name is not None:
            s.name = name
        if gpa is not None:
            s.gpa = float(gpa)
        return True

    def remove(self, student_id: str) -> bool:
        s = self.find_by_id(student_id)
        if s is None:
            return False
        self._students.remove(s)
        return True

    def search(self, keyword: str) -> List[Student]:
        kw = keyword.strip().lower()
        return [s for s in self._students if kw in s.student_id.lower() or kw in s.name.lower()]

    def sort_by_gpa(self, descending: bool = True) -> List[Student]:
        return sorted(self._students, key=lambda s: s.gpa, reverse=descending)

    def report_by_rank(self):
        from collections import Counter
        return dict(Counter(s.rank() for s in self._students))

    def list_all(self) -> List[Student]:
        return list(self._students)

mgr = StudentManager()
mgr.add(Student("SV001", "An", 3.4))
mgr.add(Student("SV002", "Bình", 2.6))
mgr.add(Student("SV003", "Chi", 3.8))

print([s.rank() for s in mgr.list_all()])
print(mgr.report_by_rank())
print([(s.student_id, s.gpa) for s in mgr.sort_by_gpa()])

---
# 9) BÀI TẬP THỰC HÀNH (kèm lời giải)

## Bài 1: Rectangle
Tạo class `Rectangle(width, height)` với `area()`, `perimeter()`, validate width/height > 0.

### Lời giải


In [ ]:
class Rectangle:
    def __init__(self, width: float, height: float):
        self.width = float(width)
        self.height = float(height)
        if self.width <= 0 or self.height <= 0:
            raise ValueError("width/height must be > 0")

    def area(self) -> float:
        return self.width * self.height

    def perimeter(self) -> float:
        return 2 * (self.width + self.height)

r = Rectangle(3, 4)
print(r.area(), r.perimeter())

## Bài 2: Account (Encapsulation)
Tạo class `Account` có balance private, deposit/withdraw, không cho rút âm.

### Lời giải


In [ ]:
class Account:
    def __init__(self, owner: str, balance: float = 0):
        self.owner = owner
        self.__balance = float(balance)

    @property
    def balance(self):
        return self.__balance

    def deposit(self, amount: float):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("amount must be positive")
        self.__balance += amount

    def withdraw(self, amount: float):
        amount = float(amount)
        if amount <= 0:
            raise ValueError("amount must be positive")
        if amount > self.__balance:
            raise ValueError("insufficient funds")
        self.__balance -= amount

acc = Account("A", 100)
acc.deposit(20)
acc.withdraw(50)
print(acc.balance)

## Bài 3: Employee (Inheritance + Polymorphism)
Tạo Employee và 2 lớp con FullTime/PartTime, viết hàm tổng lương.

### Lời giải


In [ ]:
class Employee:
    def __init__(self, name: str, base_salary: float):
        self.name = name
        self.base_salary = float(base_salary)

    def salary(self) -> float:
        raise NotImplementedError

class FullTimeEmployee(Employee):
    def salary(self) -> float:
        return self.base_salary

class PartTimeEmployee(Employee):
    def __init__(self, name: str, base_salary: float, hours: float):
        super().__init__(name, base_salary)
        self.hours = float(hours)

    def salary(self) -> float:
        return self.base_salary * self.hours

def total_salary(employees):
    return sum(e.salary() for e in employees)

emps = [FullTimeEmployee("Nam", 1000), PartTimeEmployee("Lan", 10, 80)]
print(total_salary(emps))

## Bài 4: Mở rộng StudentManager (nâng cao)
1) Thêm hàm `top_k(k=3)` trả về top k GPA.  
2) Thêm hàm `export_csv(path)` xuất danh sách ra CSV (tuỳ chọn, dùng pandas).

### Lời giải


In [ ]:
import pandas as pd

class StudentManagerV2(StudentManager):
    def top_k(self, k: int = 3):
        return self.sort_by_gpa(descending=True)[:k]

    def export_csv(self, path: str):
        df = pd.DataFrame([s.__dict__ for s in self._students])
        df.to_csv(path, index=False)

mgr2 = StudentManagerV2()
mgr2.add(Student("SV001", "An", 3.4))
mgr2.add(Student("SV002", "Bình", 2.6))
mgr2.add(Student("SV003", "Chi", 3.8))
mgr2.add(Student("SV004", "Dũng", 3.2))

print([s.student_id for s in mgr2.top_k(2)])
# mgr2.export_csv("students.csv")

---
# 10) ĐỀ KIỂM TRA NHANH (15–20 phút)
1) Phân biệt class attribute và instance attribute. Cho ví dụ.  
2) Tại sao `self` cần thiết? `self` có bắt buộc đặt tên là self không?  
3) Giải thích override là gì? Khi nào dùng `super()`?  
4) Python có hỗ trợ method overloading không? Nêu cách thay thế.  
5) Viết class `Book(title, author, price)`, có method `apply_discount(percent)`.  
